# Euler Rodrigues Parameters Functions

In [1]:
import numpy as np

import sys
from pathlib import Path

# Dynamically add DCM_utils directory to path
sys.path.insert(0, str(Path('..').resolve()))
from DCM_utils import *

# 1) EP_to_DCM

In [2]:
def EP_to_DCM(q):
    """
    Converts the EP/Quaternion to a direction cosine matrix (C).

    Args:
        q (np.array): A numpy array of size 4 (a row vector) representing the quaternion,
                      where q[0] is the scalar part (beta_0), and q[1], q[2], q[3] are the 
                      vector parts (beta_1, beta_2, beta_3).

    Returns:
        np.array: A 3x3 rotation matrix (C).
    """
    # Validate input vector
    validate_vec4(q)
    
    # Ensure q is a float array to maintain precision
    q = np.array(q, dtype=np.float64)
    
    # Check that the holonomic constraint of quaternion is satisfied, else normalize it
    q_norm = np.linalg.norm(q)
    if not np.isclose(q_norm, 1.0, atol=1e-8):
        q /= q_norm
    
    # Extract components
    q0, q1, q2, q3 = q
    
    # Compute the elements of the C
    C = np.array([
        [q0**2 + q1**2 - q2**2 - q3**2, 2 * (q1*q2 + q0*q3)          , 2 * (q1*q3 - q0*q2)          ],
        [2 * (q1*q2 - q0*q3)          , q0**2 - q1**2 + q2**2 - q3**2, 2 * (q2*q3 + q0*q1)          ],
        [2 * (q1*q3 + q0*q2)          , 2 * (q2*q3 - q0*q1)          , q0**2 - q1**2 - q2**2 + q3**2]
    ])
    
    return C

## 1.1 - Functional testing of EP_to_DCM

In [3]:
from pathlib import Path
import sys

# Add the directory where RigidBodyKinematics.py is located to sys.path
path_to_rigid_body_kinematics = Path(r"..\..\Codes from AVS Lab")
sys.path.insert(0, str(path_to_rigid_body_kinematics))

# Import the EP2C function from the AVS lab
from RigidBodyKinematics import EP2C

# Define test quaternion vectors (Euler Parameters)
test_quaternions = [
    [1, 0, 0, 0],         # Identity quaternion (no rotation)
    [0, 1, 0, 0],         # 180-degree rotation about x-axis
    [0, 0, 1, 0],         # 180-degree rotation about y-axis
    [0, 0, 0, 1],         # 180-degree rotation about z-axis
    [np.sqrt(0.5), np.sqrt(0.5), 0, 0],  # 90-degree rotation about x-axis
    [np.sqrt(0.5), 0, np.sqrt(0.5), 0],  # 90-degree rotation about y-axis
    [np.sqrt(0.5), 0, 0, np.sqrt(0.5)],  # 90-degree rotation about z-axis
]

# Test each quaternion
for i, q in enumerate(test_quaternions):
    q = np.array(q, dtype=float)

    print(f"Test Case {i + 1}:")
    print(f"Quaternion (q) = {q}")

    # Compute DCM using the existing EP2C function
    C_existing = EP2C(q)

    # Compute DCM using your EP_to_DCM function
    C_custom = EP_to_DCM(q)

    # Ensure both are NumPy arrays for easy comparison
    C_existing = np.array(C_existing)
    C_custom = np.array(C_custom)

    # Calculate the difference between the two C matrices
    difference = C_existing - C_custom
    max_diff = np.max(np.abs(difference))

    # Print the results
    print(f"Max difference between EP_to_DCM and EP2C: {max_diff:.12e}")
    if max_diff > 1e-12:
        print("C matrices differ significantly.\n")
    else:
        print("C matrices match.\n")

    print("-" * 50)


Test Case 1:
Quaternion (q) = [1. 0. 0. 0.]
Max difference between EP_to_DCM and EP2C: 0.000000000000e+00
C matrices match.

--------------------------------------------------
Test Case 2:
Quaternion (q) = [0. 1. 0. 0.]
Max difference between EP_to_DCM and EP2C: 0.000000000000e+00
C matrices match.

--------------------------------------------------
Test Case 3:
Quaternion (q) = [0. 0. 1. 0.]
Max difference between EP_to_DCM and EP2C: 0.000000000000e+00
C matrices match.

--------------------------------------------------
Test Case 4:
Quaternion (q) = [0. 0. 0. 1.]
Max difference between EP_to_DCM and EP2C: 0.000000000000e+00
C matrices match.

--------------------------------------------------
Test Case 5:
Quaternion (q) = [0.70710678 0.70710678 0.         0.        ]
Max difference between EP_to_DCM and EP2C: 0.000000000000e+00
C matrices match.

--------------------------------------------------
Test Case 6:
Quaternion (q) = [0.70710678 0.         0.70710678 0.        ]
Max differen

# 2) DCM_to_EP

In [4]:
def DCM_to_EP(C):
    """
    Converts a Direction Cosine Matrix (C) to a quaternion using Shepperd's method to ensure robustness against numerical issues.
    
    Args:
        C (np.array): A 3x3 rotation matrix (C).
    
    Returns:
        np.array: A quaternion represented as a numpy array of size 4, with the scalar component as the first element.
    """
    trace = np.trace(C)
    q_squared = np.zeros(4)
    q_squared[0] = (1.0 + trace) / 4.0
    q_squared[1] = (1.0 + 2 * C[0, 0] - trace) / 4.0
    q_squared[2] = (1.0 + 2 * C[1, 1] - trace) / 4.0
    q_squared[3] = (1.0 + 2 * C[2, 2] - trace) / 4.0

    q = np.zeros(4)
    max_index = np.argmax(q_squared)

    if max_index == 0:
        q[0] = np.sqrt(q_squared[0])
        q[1] = (C[1, 2] - C[2, 1]) / (4 * q[0])
        q[2] = (C[2, 0] - C[0, 2]) / (4 * q[0])
        q[3] = (C[0, 1] - C[1, 0]) / (4 * q[0])
    
    elif max_index == 1:
        q[1] = np.sqrt(q_squared[1])
        q[0] = (C[1, 2] - C[2, 1]) / (4 * q[1])
        if q[0] < 0:
            q[0] = -q[0]
            q[1] = -q[1]
        q[2] = (C[0, 1] + C[1, 0]) / (4 * q[1])
        q[3] = (C[2, 0] + C[0, 2]) / (4 * q[1])
        
    elif max_index == 2:
        q[2] = np.sqrt(q_squared[2])
        q[0] = (C[2, 0] - C[0, 2]) / (4 * q[2])
        if q[0] < 0:
            q[0] = -q[0]
            q[2] = -q[2]
        q[1] = (C[0, 1] + C[1, 0]) / (4 * q[2])
        q[3] = (C[1, 2] + C[2, 1]) / (4 * q[2])

    elif max_index == 3:
        q[3] = np.sqrt(q_squared[3])
        q[0] = (C[0, 1] - C[1, 0]) / (4 * q[3])
        if q[0] < 0:
            q[0] = -q[0]
            q[3] = -q[3]
        q[1] = (C[2, 0] + C[0, 2]) / (4 * q[3])
        q[2] = (C[1, 2] + C[2, 1]) / (4 * q[3])
    
    return q

## 2.1 - Functional testing of DCM_to_EP

In [5]:
# Define test quaternions (Euler Parameters)
test_quaternions = [
    [1, 0, 0, 0],                                                  # Identity quaternion (no rotation)
    [0, 1, 0, 0],                                                  # 180-degree rotation about x-axis
    [0, 0, 1, 0],                                                  # 180-degree rotation about y-axis
    [0, 0, 0, 1],                                                  # 180-degree rotation about z-axis
    [np.sqrt(0.5), np.sqrt(0.5), 0, 0],                            # 90-degree rotation about x-axis
    [np.sqrt(0.5), 0, np.sqrt(0.5), 0],                            # 90-degree rotation about y-axis
    [np.sqrt(0.5), 0, 0, np.sqrt(0.5)],                            # 90-degree rotation about z-axis
    [np.sqrt(0.25), np.sqrt(0.25), np.sqrt(0.25), np.sqrt(0.25)],  # General rotation
]

# Test each quaternion
for i, q in enumerate(test_quaternions):
    q = np.array(q, dtype=float)
    print(f"Test Case {i + 1}:")
    print(f"Input Quaternion (q) = {q}")

    # Normalize quaternion to ensure it represents a valid rotation
    q_normalized = q / np.linalg.norm(q)

    # Compute DCM from quaternion using EP_to_DCM function
    C = EP_to_DCM(q_normalized)

    # Get quaternion back from DCM using DCM_to_EP function
    q_reconstructed = DCM_to_EP(C)

    # Normalize reconstructed quaternion to ensure valid comparison
    q_reconstructed_normalized = q_reconstructed / np.linalg.norm(q_reconstructed)

    # Calculate the difference between the original and reconstructed quaternions
    difference = q_normalized - q_reconstructed_normalized
    max_diff = np.max(np.abs(difference))

    # Print the results
    print(f"Reconstructed Quaternion (q) = {q_reconstructed_normalized}")
    print(f"Max difference: {max_diff:.12e}")

    if max_diff > 1e-12:
        print("Quaternions differ significantly.\n")
    else:
        print("Quaternions match.\n")

    print("-" * 50)


Test Case 1:
Input Quaternion (q) = [1. 0. 0. 0.]
Reconstructed Quaternion (q) = [1. 0. 0. 0.]
Max difference: 0.000000000000e+00
Quaternions match.

--------------------------------------------------
Test Case 2:
Input Quaternion (q) = [0. 1. 0. 0.]
Reconstructed Quaternion (q) = [0. 1. 0. 0.]
Max difference: 0.000000000000e+00
Quaternions match.

--------------------------------------------------
Test Case 3:
Input Quaternion (q) = [0. 0. 1. 0.]
Reconstructed Quaternion (q) = [0. 0. 1. 0.]
Max difference: 0.000000000000e+00
Quaternions match.

--------------------------------------------------
Test Case 4:
Input Quaternion (q) = [0. 0. 0. 1.]
Reconstructed Quaternion (q) = [0. 0. 0. 1.]
Max difference: 0.000000000000e+00
Quaternions match.

--------------------------------------------------
Test Case 5:
Input Quaternion (q) = [0.70710678 0.70710678 0.         0.        ]
Reconstructed Quaternion (q) = [0.70710678 0.70710678 0.         0.        ]
Max difference: 1.110223024625e-16
Qu

# 3) Bmat_EP

In [6]:
def Bmat_EP(q):
    """
    Computes the 4x3 B matrix that maps body angular velocity (omega) to the derivative of the quaternion (Euler parameters) vector.

        dQ/dt = 1/2 * [B(Q)] * omega

    Args:
        q (array-like): A 4-element quaternion (Euler parameter) vector [q0, q1, q2, q3].
    
    Returns:
        np.ndarray: A 4x3 B matrix.
    
    Notes:
        - The quaternion vector q should be in the form [q0, q1, q2, q3], where q0 is the scalar component.
    """
    # Validate the input quaternion vector
    validate_vec4(q)

    # Convert input to a NumPy array if not already
    q = np.array(q, dtype=float)

    # Extract components of the quaternion
    q0, q1, q2, q3 = q

    # Construct the B matrix using a structured array
    B = np.array([[-q1, -q2, -q3],
                  [ q0, -q3,  q2],
                  [ q3,  q0, -q1],
                  [-q2,  q1,  q0]])

    return B

Bmat_EP([0,1,2,1])

array([[-1., -2., -1.],
       [ 0., -1.,  2.],
       [ 1.,  0., -1.],
       [-2.,  1.,  0.]])

## 3.1 - Functional testing of Bmat_EP

In [7]:
from pathlib import Path
import sys

# Add the directory where RigidBodyKinematics.py is located to sys.path
path_to_rigid_body_kinematics = Path(r"..\..\Codes from AVS Lab")
sys.path.insert(0, str(path_to_rigid_body_kinematics))

# Import the BmatEP function from the AVS lab
from RigidBodyKinematics import BmatEP

# Define test quaternion vectors (Euler Parameters)
test_quaternions = [
    [1, 0, 0, 0],                        # Identity quaternion (no rotation)
    [0, 1, 0, 0],                        # 180-degree rotation about x-axis
    [0, 0, 1, 0],                        # 180-degree rotation about y-axis
    [0, 0, 0, 1],                        # 180-degree rotation about z-axis
    [np.sqrt(0.5), np.sqrt(0.5), 0, 0],  # 90-degree rotation about x-axis
    [np.sqrt(0.5), 0, np.sqrt(0.5), 0],  # 90-degree rotation about y-axis
    [np.sqrt(0.5), 0, 0, np.sqrt(0.5)],  # 90-degree rotation about z-axis
]

# Test each quaternion
for i, q in enumerate(test_quaternions):
    q = np.array(q, dtype=float)
    print(f"Test Case {i + 1}:")
    print(f"Quaternion (q) = {q}")

    # Compute B matrix using the existing BmatEP function
    B_existing = BmatEP(q)

    # Compute B matrix using your Bmat_EP function
    B_custom = Bmat_EP(q)

    # Ensure both are NumPy arrays for easy comparison
    B_existing = np.array(B_existing)
    B_custom = np.array(B_custom)

    # Calculate the difference between the two B matrices
    difference = B_existing - B_custom
    max_diff = np.max(np.abs(difference))

    # Print the results
    print(f"Max difference between Bmat_EP and BmatEP: {max_diff:.12e}")
    if max_diff > 1e-12:
        print("B matrices differ significantly.\n")
    else:
        print("B matrices match.\n")

    print("-" * 50)


Test Case 1:
Quaternion (q) = [1. 0. 0. 0.]
Max difference between Bmat_EP and BmatEP: 0.000000000000e+00
B matrices match.

--------------------------------------------------
Test Case 2:
Quaternion (q) = [0. 1. 0. 0.]
Max difference between Bmat_EP and BmatEP: 0.000000000000e+00
B matrices match.

--------------------------------------------------
Test Case 3:
Quaternion (q) = [0. 0. 1. 0.]
Max difference between Bmat_EP and BmatEP: 0.000000000000e+00
B matrices match.

--------------------------------------------------
Test Case 4:
Quaternion (q) = [0. 0. 0. 1.]
Max difference between Bmat_EP and BmatEP: 0.000000000000e+00
B matrices match.

--------------------------------------------------
Test Case 5:
Quaternion (q) = [0.70710678 0.70710678 0.         0.        ]
Max difference between Bmat_EP and BmatEP: 0.000000000000e+00
B matrices match.

--------------------------------------------------
Test Case 6:
Quaternion (q) = [0.70710678 0.         0.70710678 0.        ]
Max differen

# 4) BInvmat_EP

In [8]:
def BInvmat_EP(q):
    """
    Computes the 3x4 B matrix that maps the derivative of the quaternion (Euler parameters) vector to the body angular velocity (omega).

        omega = 2 * [B(Q)]^(-1) * dQ/dt

    Args:
        q (array-like): A 4-element quaternion (Euler parameter) vector [q0, q1, q2, q3].

    Returns:
        np.ndarray: A 3x4 B matrix.
    
    Notes:
        - The quaternion vector q should be in the form [q0, q1, q2, q3], where q0 is the scalar component.
        - This matrix is used to map quaternion rates to body angular velocity.
    """
    # Validate the input quaternion vector
    validate_vec4(q)

    # Convert input to a NumPy array if not already
    q = np.array(q, dtype=float)

    # Extract components of the quaternion
    q0, q1, q2, q3 = q

    # Construct the BInv matrix using a structured array
    B_inv = np.array([[-q1,  q0,  q3, -q2],
                      [-q2, -q3,  q0,  q1],
                      [-q3,  q2, -q1,  q0]])

    return B_inv
    
BInvmat_EP([0,1,2,1])

array([[-1.,  0.,  1., -2.],
       [-2., -1.,  0.,  1.],
       [-1.,  2., -1.,  0.]])

## 4.1 - Functional testing of BInvmat_EP

In [9]:
from pathlib import Path
import sys
import numpy as np

# Add the directory where RigidBodyKinematics.py is located to sys.path
path_to_rigid_body_kinematics = Path(r"..\..\Codes from AVS Lab")
sys.path.insert(0, str(path_to_rigid_body_kinematics))

# Import the BinvEP function from the AVS lab
from RigidBodyKinematics import BinvEP

# Define test quaternion vectors (Euler Parameters)
test_quaternions = [
    [1, 0, 0, 0],                        # Identity quaternion (no rotation)
    [0, 1, 0, 0],                        # 180-degree rotation about x-axis
    [0, 0, 1, 0],                        # 180-degree rotation about y-axis
    [0, 0, 0, 1],                        # 180-degree rotation about z-axis
    [np.sqrt(0.5), np.sqrt(0.5), 0, 0],  # 90-degree rotation about x-axis
    [np.sqrt(0.5), 0, np.sqrt(0.5), 0],  # 90-degree rotation about y-axis
    [np.sqrt(0.5), 0, 0, np.sqrt(0.5)],  # 90-degree rotation about z-axis
]

# Test each quaternion
for i, q in enumerate(test_quaternions):
    q = np.array(q, dtype=float)
    print(f"Test Case {i + 1}:")
    print(f"Quaternion (q) = {q}")

    # Compute B inverse matrix using the existing BinvEP function
    B_existing = BinvEP(q)

    # Compute B inverse matrix using your BInvmat_EP function
    B_custom = BInvmat_EP(q)

    # Ensure both are NumPy arrays for easy comparison
    B_existing = np.array(B_existing)
    B_custom = np.array(B_custom)

    # Calculate the difference between the two B matrices
    difference = B_existing - B_custom
    max_diff = np.max(np.abs(difference))

    # Print the results
    print(f"Max difference between BInvmat_EP and BinvEP: {max_diff:.12e}")
    if max_diff > 1e-12:
        print("B inverse matrices differ significantly.\n")
    else:
        print("B inverse matrices match.\n")

    print("-" * 50)


Test Case 1:
Quaternion (q) = [1. 0. 0. 0.]
Max difference between BInvmat_EP and BinvEP: 0.000000000000e+00
B inverse matrices match.

--------------------------------------------------
Test Case 2:
Quaternion (q) = [0. 1. 0. 0.]
Max difference between BInvmat_EP and BinvEP: 0.000000000000e+00
B inverse matrices match.

--------------------------------------------------
Test Case 3:
Quaternion (q) = [0. 0. 1. 0.]
Max difference between BInvmat_EP and BinvEP: 0.000000000000e+00
B inverse matrices match.

--------------------------------------------------
Test Case 4:
Quaternion (q) = [0. 0. 0. 1.]
Max difference between BInvmat_EP and BinvEP: 0.000000000000e+00
B inverse matrices match.

--------------------------------------------------
Test Case 5:
Quaternion (q) = [0.70710678 0.70710678 0.         0.        ]
Max difference between BInvmat_EP and BinvEP: 0.000000000000e+00
B inverse matrices match.

--------------------------------------------------
Test Case 6:
Quaternion (q) = [0.